In [1]:
%config Completer.use_jedi = True
from sklearn.linear_model import LogisticRegression
import numpy as np
import math
from tqdm import tqdm
import pandas as pd
import sklearn
import seaborn as sns
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, ConfusionMatrixDisplay, confusion_matrix
from sklearn.model_selection import RandomizedSearchCV
sns.set(style="darkgrid",rc={'figure.figsize':(15,8)})

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import numpy as np

In [2]:
# identifico los nans de cada feature con un string para luego tener la feature de isnan al hacer onehot encoding
def nan_imputation_cat(df, feature_names):
    categoricals = {'DeviceType' : 'nan_devicetype', 'DeviceInfo' : 'nan_deviceinfo', 'card4' : 'nan_card4', 'card6' : 'nan_card6', 'id_31' : 'nan_id_31', 'id_30' : 'nan_id_30',
                    'id_33' : 'nan_id_33', 'P_emaildomain' : 'nan_P_emaildomain', 'R_emaildomain' : 'nan_R_emaildomain', 'addr1' : 'nan_addr1', 'addr2' : 'nan_addr2'}
    for feature in feature_names:
        df[feature].loc[df[feature].isna()] = categoricals[feature] #si rompe aca es porque falta agregar la feature al diccionario

In [3]:
# imputo nans agregando features del estilo isnan para train y me guardo que features para los nans deben tener validation y test
class nan_imputation_num:
    def __init__(self):
        self.columns = []
    
    def fit_imputate(self, df):
        for col in df.columns:
            if df[col].hasnans:
                df['nan_'+col] = df[col].isna().apply(lambda x : int(x))
                df[col].fillna(0, inplace=True)
                self.columns.append(col)
                
    def imputate(self, df):
        for col in self.columns:
            if df[col].hasnans:
                df['nan_'+col] = df[col].isna().apply(lambda x : int(x))
                df[col].fillna(0, inplace=True)
            else:
                df['nan_'+col] = 0

In [4]:
def mean_encoder(df, featureNames, targetName):
    dict_mappings = {}
    for feature in featureNames:
        Mean_encoded_feature = df.groupby([feature])[targetName].mean().to_dict()
        df[feature] =  df[feature].map(Mean_encoded_feature).astype(float, errors = 'raise')
        dict_mappings[feature] = Mean_encoded_feature
    return dict_mappings

# FEATURE ENGINEERING

# Seleccion de features

In [5]:
"""
oldTrain = pd.read_csv('trainSet.csv').drop(columns='TransactionID')

featuresPart2 = pd.read_csv('FeatureImportanceLogReg.csv').head(40)

featuresRandFor = pd.read_csv('FeatureImportanceRandFor.csv')

train = oldTrain.loc[:, featuresRandFor.feature]

train = train.join(oldTrain.loc[:, featuresPart2.feature].drop(columns=['DeviceInfo', 'V258', 'V201', 'V257', 'V246', 'V199']))

train = train.join(oldTrain.loc[:, ['DeviceType', 'id_33', 'card3', 'id_30', 'addr2', 'id_14', 'isFraud']])

train.to_csv("trainPart3.csv",index=False)
"""

'\noldTrain = pd.read_csv(\'trainSet.csv\').drop(columns=\'TransactionID\')\n\nfeaturesPart2 = pd.read_csv(\'FeatureImportanceLogReg.csv\').head(40)\n\nfeaturesRandFor = pd.read_csv(\'FeatureImportanceRandFor.csv\')\n\ntrain = oldTrain.loc[:, featuresRandFor.feature]\n\ntrain = train.join(oldTrain.loc[:, featuresPart2.feature].drop(columns=[\'DeviceInfo\', \'V258\', \'V201\', \'V257\', \'V246\', \'V199\']))\n\ntrain = train.join(oldTrain.loc[:, [\'DeviceType\', \'id_33\', \'card3\', \'id_30\', \'addr2\', \'id_14\', \'isFraud\']])\n\ntrain.to_csv("trainPart3.csv",index=False)\n'

In [6]:
train = pd.read_csv('data/Parte3/trainPart3.csv')
train

,TransactionDT,TransactionAmt,card1,card2,addr1,C13,DeviceInfo,P_emaildomain,C1,card5,...,V28,V27,V331,DeviceType,id_33,card3,id_30,addr2,id_14,isFraud
0,86400,68.50,13926,NaN,315.0,1.0,NaN,NaN,1.0,142.0,...,0.0,0.0,NaN,NaN,NaN,150.0,NaN,87.0,NaN,0
1,86401,29.00,2755,404.0,325.0,1.0,NaN,gmail.com,1.0,102.0,...,0.0,0.0,NaN,NaN,NaN,150.0,NaN,87.0,NaN,0
2,86469,59.00,4663,490.0,330.0,1.0,NaN,outlook.com,1.0,166.0,...,0.0,0.0,NaN,NaN,NaN,150.0,NaN,87.0,NaN,0
3,86499,50.00,18132,567.0,476.0,25.0,NaN,yahoo.com,2.0,117.0,...,0.0,0.0,NaN,NaN,NaN,150.0,NaN,87.0,NaN,0
4,86506,50.00,4497,514.0,420.0,1.0,SAMSUNG SM-G892A Build/NRD90M,gmail.com,1.0,102.0,...,NaN,NaN,0.0,mobile,2220x1080,150.0,Android 7.0,87.0,-480.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590535,15811047,49.00,6550,NaN,272.0,3.0,NaN,NaN,2.0,226.0,...,0.0,0.0,NaN,NaN,NaN,150.0,NaN,87.0,NaN,0
590536,15811049,39.50,10444,225.0,204.0,1.0,NaN,gmail.com,1.0,224.0,...,0.0,0.0,NaN,NaN,NaN,150.0,NaN,87.0,NaN,0
590537,15811079,30.95,12037,595.0,231.0,1.0,NaN,gmail.com,1.0,224.0,...,0.0,0.0,NaN,NaN,NaN,150.0,NaN,87.0,NaN,0
590538,15811088,117.00,7826,481.0,387.0,5.0,NaN,aol.com,1.0,224.0,...,0.0,0.0,NaN,NaN,NaN,150.0,NaN,87.0,NaN,0


In [7]:
#test = (pd.read_csv('data/test_transaction.csv').drop(columns='TransactionID')).join(pd.read_csv('data/test_identity.csv').drop(columns='TransactionID'))

In [8]:
#test.rename(inplace=True, columns={"id-01":"id_01", "id-02":"id_02","id-03":"id_03","id-04":"id_04","id-05":"id_05","id-06":"id_06","id-07":"id_07","id-08":"id_08","id-09":"id_09","id-11":"id_11","id-12":"id_12","id-13":"id_13","id-14":"id_14","id-15":"id_15","id-16":"id_16","id-17":"id_17","id-18":"id_18","id-19":"id_19","id-20":"id_20","id-21":"id_21","id-22":"id_22","id-23":"id_23","id-24":"id_24","id-25":"id_25","id-26":"id_26","id-27":"id_27","id-28":"id_28","id-29":"id_29","id-30":"id_30","id-31":"id_31","id-32":"id_32","id-33":"id_33","id-34":"id_34","id-35":"id_35","id-36":"id_36","id-37":"id_37","id-38":"id_38",})

In [9]:
nan_imputation_cat(train, ['DeviceType', 'DeviceInfo', 'card4', 'card6', 'id_31', 'id_33', 'id_30', 'P_emaildomain', 'R_emaildomain', 'addr1', 'addr2'])

/mnt/882C57D72C57BEBE/Anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [10]:
"""
features = train.columns
for feature in test.columns:
    if feature not in features:
        test.drop(columns=feature, inplace=True)
"""

'\nfeatures = train.columns\nfor feature in test.columns:\n    if feature not in features:\n        test.drop(columns=feature, inplace=True)\n'

In [11]:
#test.to_csv('testPart3.csv', index=False)

In [12]:
test = pd.read_csv('testPart3.csv')

In [13]:
nan_imputation_cat(test, ['DeviceType', 'DeviceInfo', 'card4', 'card6', 'id_31', 'id_33', 'id_30', 'P_emaildomain', 'R_emaildomain', 'addr1', 'addr2'])

In [14]:
# separo el 10% del train para usarlo de validation
fechaCorte = int((train['TransactionDT'].max() - train['TransactionDT'].min()) *0.9) + train['TransactionDT'].min()
validationSetX = train.loc[train['TransactionDT'] > fechaCorte]
validationSetY = validationSetX['isFraud']
validationSetX.drop(columns='isFraud', inplace=True)

/mnt/882C57D72C57BEBE/Anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


# Mean Encodings

In [15]:
emailDomainsMappings = mean_encoder(train, ['P_emaildomain', 'R_emaildomain', 'addr1', 'addr2'], 'isFraud')
# encoding de validationX
for feature, mapsDict in emailDomainsMappings.items():
    validationSetX[feature] = validationSetX[feature].map(mapsDict)

# encoding de test
for feature, mapsDict in emailDomainsMappings.items():
    test[feature] = test[feature].map(mapsDict)

<ipython-input-15-b8b51e22307c>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  validationSetX[feature] = validationSetX[feature].map(mapsDict)


In [16]:
# termino de separar el train después de haber hecho mean encoding
trainX = train.loc[train['TransactionDT'] <= fechaCorte]
trainY = trainX['isFraud']
trainX.drop(columns='isFraud', inplace=True)
del train

/mnt/882C57D72C57BEBE/Anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [17]:
imputator_num = nan_imputation_num()
imputator_num.fit_imputate(trainX)
trainX

<ipython-input-3-c05be69fdc22>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['nan_'+col] = df[col].isna().apply(lambda x : int(x))
/mnt/882C57D72C57BEBE/Anaconda3/lib/python3.8/site-packages/pandas/core/series.py:4517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


,TransactionDT,TransactionAmt,card1,card2,addr1,C13,DeviceInfo,P_emaildomain,C1,card5,...,nan_V171,nan_V229,nan_V268,nan_V230,nan_V259,nan_V28,nan_V27,nan_V331,nan_card3,nan_id_14
0,86400,68.500,13926,0.0,0.017809,1.0,nan_deviceinfo,0.029538,1.0,142.0,...,1,1,1,1,1,0,0,1,0,1
1,86401,29.000,2755,404.0,0.025426,1.0,nan_deviceinfo,0.043542,1.0,102.0,...,1,1,1,1,1,0,0,1,0,1
2,86469,59.000,4663,490.0,0.031955,1.0,nan_deviceinfo,0.094584,1.0,166.0,...,1,1,1,1,1,0,0,1,0,1
3,86499,50.000,18132,567.0,0.031652,25.0,nan_deviceinfo,0.022757,2.0,117.0,...,1,1,1,1,1,0,0,1,0,1
4,86506,50.000,4497,514.0,0.032672,1.0,SAMSUNG SM-G892A Build/NRD90M,0.043542,1.0,102.0,...,0,0,0,0,0,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539999,14238592,59.000,14128,170.0,0.025426,18.0,nan_deviceinfo,0.022757,1.0,219.0,...,1,1,1,1,1,0,0,1,0,1
540000,14238604,13.772,5812,408.0,0.117813,4.0,Moto G (5) Plus Build/NPN25.137-92,0.052950,4.0,224.0,...,0,0,0,0,0,0,0,1,0,1
540001,14238622,26.191,15944,266.0,0.117813,1.0,SM-J701M Build/NRD90M,0.043542,1.0,137.0,...,0,1,1,1,0,0,0,1,0,1
540002,14238636,57.950,17188,321.0,0.024986,2.0,nan_deviceinfo,0.022757,2.0,226.0,...,1,1,1,1,1,0,0,1,0,1


In [18]:
imputator_num.imputate(validationSetX)
validationSetX

<ipython-input-3-c05be69fdc22>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['nan_'+col] = df[col].isna().apply(lambda x : int(x))
/mnt/882C57D72C57BEBE/Anaconda3/lib/python3.8/site-packages/pandas/core/series.py:4517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
<ipython-input-3-c05be69fdc22>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gu

,TransactionDT,TransactionAmt,card1,card2,addr1,C13,DeviceInfo,P_emaildomain,C1,card5,...,nan_V171,nan_V229,nan_V268,nan_V230,nan_V259,nan_V28,nan_V27,nan_V331,nan_card3,nan_id_14
540004,14238660,36.95,7207,111.0,0.026654,63.0,nan_deviceinfo,0.022757,9.0,226.0,...,1,1,1,1,1,0,0,1,0,1
540005,14238667,21.95,18132,567.0,0.016623,2.0,nan_deviceinfo,0.031187,1.0,117.0,...,1,1,1,1,1,0,0,1,0,1
540006,14238672,49.00,17321,111.0,0.026654,2.0,nan_deviceinfo,0.022757,1.0,226.0,...,1,1,1,1,1,0,0,1,0,1
540007,14238694,67.95,14181,555.0,0.038833,73.0,nan_deviceinfo,0.021811,10.0,224.0,...,1,1,1,1,1,0,0,1,0,1
540008,14238717,24.00,13780,298.0,0.025592,63.0,nan_deviceinfo,0.029538,1.0,226.0,...,1,1,1,1,1,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590535,15811047,49.00,6550,0.0,0.028598,3.0,nan_deviceinfo,0.029538,2.0,226.0,...,1,1,1,1,1,0,0,1,0,1
590536,15811049,39.50,10444,225.0,0.026654,1.0,nan_deviceinfo,0.043542,1.0,224.0,...,1,1,1,1,1,0,0,1,0,1
590537,15811079,30.95,12037,595.0,0.028665,1.0,nan_deviceinfo,0.043542,1.0,224.0,...,1,1,1,1,1,0,0,1,0,1
590538,15811088,117.00,7826,481.0,0.019177,5.0,nan_deviceinfo,0.021811,1.0,224.0,...,1,1,1,1,1,0,0,1,0,1


In [19]:
imputator_num.imputate(test)
test

,TransactionDT,TransactionAmt,card1,card2,card3,card4,card5,card6,addr1,addr2,...,nan_V171,nan_V229,nan_V268,nan_V230,nan_V259,nan_V28,nan_V27,nan_V331,nan_card3,nan_id_14
0,18403224,31.950,10409,111.0,150.0,visa,226.0,debit,NaN,NaN,...,1,1,1,1,1,0,0,1,0,1
1,18403263,49.000,4272,111.0,150.0,visa,226.0,debit,NaN,NaN,...,1,1,1,1,1,0,0,1,0,0
2,18403310,171.000,4476,574.0,150.0,visa,226.0,debit,NaN,NaN,...,1,1,1,1,1,0,0,1,0,0
3,18403310,284.950,10989,360.0,150.0,visa,166.0,debit,NaN,NaN,...,1,1,1,1,1,0,0,1,0,1
4,18403317,67.950,18018,452.0,150.0,mastercard,117.0,debit,NaN,NaN,...,1,1,1,1,1,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
506686,34214279,94.679,13832,375.0,185.0,mastercard,224.0,debit,NaN,NaN,...,1,1,1,1,1,0,0,1,0,1
506687,34214287,12.173,3154,408.0,185.0,mastercard,224.0,debit,0.117813,0.117813,...,0,0,0,0,0,0,0,1,0,1
506688,34214326,49.000,16661,490.0,150.0,visa,226.0,debit,NaN,NaN,...,1,1,1,1,1,0,0,1,0,1
506689,34214337,202.000,16621,516.0,150.0,mastercard,224.0,debit,NaN,NaN,...,1,1,1,1,1,0,0,1,0,1


# Bag Of Words

In [20]:
def bagOfWords(trainX, validationSetX, test):
    features = ['DeviceInfo', 'id_31', 'id_33', 'id_30']
    for feature in features:
        vect = CountVectorizer(max_features=30)
        matrix = vect.fit_transform(trainX[feature])
        trainX = trainX.join(pd.DataFrame(matrix.toarray(),columns=vect.get_feature_names_out()), rsuffix='_'+feature)
        trainX.drop(columns=[feature],inplace=True)
        
        matrixVal = vect.transform(validationSetX[feature])
        validationSetX = validationSetX.join(pd.DataFrame(matrixVal.toarray(),columns=vect.get_feature_names_out()).set_index(validationSetX.index), rsuffix='_'+feature)
        validationSetX.drop(columns=[feature],inplace=True)
        
        matrixtest = vect.transform(test[feature])
        test = test.join(pd.DataFrame(matrixtest.toarray(),columns=vect.get_feature_names_out()).set_index(test.index), rsuffix='_'+feature)
        test.drop(columns=[feature],inplace=True)
    return trainX, validationSetX, test

In [21]:
trainX, validationSetX, test = bagOfWords(trainX, validationSetX, test)

In [22]:
trainX

,TransactionDT,TransactionAmt,card1,card2,addr1,C13,P_emaildomain,C1,card5,C6,...,13,android_id_30,ios_id_30,linux,mac,nan_id_30,os,vista,windows_id_30,xp
0,86400,68.500,13926,0.0,0.017809,1.0,0.029538,1.0,142.0,1.0,...,0,0,0,0,0,1,0,0,0,0
1,86401,29.000,2755,404.0,0.025426,1.0,0.043542,1.0,102.0,1.0,...,0,0,0,0,0,1,0,0,0,0
2,86469,59.000,4663,490.0,0.031955,1.0,0.094584,1.0,166.0,1.0,...,0,0,0,0,0,1,0,0,0,0
3,86499,50.000,18132,567.0,0.031652,25.0,0.022757,2.0,117.0,4.0,...,0,0,0,0,0,1,0,0,0,0
4,86506,50.000,4497,514.0,0.032672,1.0,0.043542,1.0,102.0,1.0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539999,14238592,59.000,14128,170.0,0.025426,18.0,0.022757,1.0,219.0,1.0,...,0,0,0,0,0,1,0,0,0,0
540000,14238604,13.772,5812,408.0,0.117813,4.0,0.052950,4.0,224.0,2.0,...,0,0,0,0,0,1,0,0,0,0
540001,14238622,26.191,15944,266.0,0.117813,1.0,0.043542,1.0,137.0,1.0,...,0,0,0,0,0,1,0,0,0,0
540002,14238636,57.950,17188,321.0,0.024986,2.0,0.022757,2.0,226.0,2.0,...,0,0,0,0,0,1,0,0,0,0


In [23]:
validationSetX

,TransactionDT,TransactionAmt,card1,card2,addr1,C13,P_emaildomain,C1,card5,C6,...,13,android_id_30,ios_id_30,linux,mac,nan_id_30,os,vista,windows_id_30,xp
540004,14238660,36.95,7207,111.0,0.026654,63.0,0.022757,9.0,226.0,6.0,...,0,0,0,0,0,1,0,0,0,0
540005,14238667,21.95,18132,567.0,0.016623,2.0,0.031187,1.0,117.0,1.0,...,0,0,0,0,0,1,0,0,0,0
540006,14238672,49.00,17321,111.0,0.026654,2.0,0.022757,1.0,226.0,2.0,...,0,0,0,0,0,1,0,0,0,0
540007,14238694,67.95,14181,555.0,0.038833,73.0,0.021811,10.0,224.0,5.0,...,0,0,0,0,0,1,0,0,0,0
540008,14238717,24.00,13780,298.0,0.025592,63.0,0.029538,1.0,226.0,0.0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590535,15811047,49.00,6550,0.0,0.028598,3.0,0.029538,2.0,226.0,0.0,...,0,0,0,0,0,1,0,0,0,0
590536,15811049,39.50,10444,225.0,0.026654,1.0,0.043542,1.0,224.0,1.0,...,0,0,0,0,0,1,0,0,0,0
590537,15811079,30.95,12037,595.0,0.028665,1.0,0.043542,1.0,224.0,1.0,...,0,0,0,0,0,1,0,0,0,0
590538,15811088,117.00,7826,481.0,0.019177,5.0,0.021811,1.0,224.0,3.0,...,0,0,0,0,0,1,0,0,0,0


In [24]:
test

,TransactionDT,TransactionAmt,card1,card2,card3,card4,card5,card6,addr1,addr2,...,13,android_id_30,ios_id_30,linux,mac,nan_id_30,os,vista,windows_id_30,xp
0,18403224,31.950,10409,111.0,150.0,visa,226.0,debit,NaN,NaN,...,0,0,0,0,0,1,0,0,0,0
1,18403263,49.000,4272,111.0,150.0,visa,226.0,debit,NaN,NaN,...,0,1,0,0,0,0,0,0,0,0
2,18403310,171.000,4476,574.0,150.0,visa,226.0,debit,NaN,NaN,...,0,0,0,0,0,1,0,0,0,0
3,18403310,284.950,10989,360.0,150.0,visa,166.0,debit,NaN,NaN,...,0,0,0,0,0,1,0,0,0,0
4,18403317,67.950,18018,452.0,150.0,mastercard,117.0,debit,NaN,NaN,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
506686,34214279,94.679,13832,375.0,185.0,mastercard,224.0,debit,NaN,NaN,...,0,0,0,0,0,1,0,0,0,0
506687,34214287,12.173,3154,408.0,185.0,mastercard,224.0,debit,0.117813,0.117813,...,0,0,0,0,0,1,0,0,0,0
506688,34214326,49.000,16661,490.0,150.0,visa,226.0,debit,NaN,NaN,...,0,0,0,0,0,1,0,0,0,0
506689,34214337,202.000,16621,516.0,150.0,mastercard,224.0,debit,NaN,NaN,...,0,0,0,0,0,1,0,0,0,0


# OneHot Encodings

In [25]:
def oneHot(trainX, validationSetX, test):
    features = ['DeviceType', 'card4', 'card6']
    for feature in features:
        onehot = OneHotEncoder(sparse=False)
        matrix = pd.DataFrame(data=onehot.fit_transform(trainX[[feature]]), columns=onehot.categories_)
        matrix.columns = matrix.columns.map(lambda x : x[0])
        trainX = trainX.join(matrix, rsuffix='_'+feature)
        trainX.drop(columns=[feature],inplace=True)

        matrix = pd.DataFrame(data=onehot.transform(validationSetX[[feature]]), columns=onehot.categories_).set_index(validationSetX.index)
        matrix.columns = matrix.columns.map(lambda x : x[0])
        validationSetX.drop(columns=[feature],inplace=True)
        validationSetX = validationSetX.join(matrix, rsuffix='_'+feature)

        matrix = pd.DataFrame(data=onehot.transform(test[[feature]]), columns=onehot.categories_).set_index(test.index)
        matrix.columns = matrix.columns.map(lambda x : x[0])
        test.drop(columns=[feature],inplace=True)
        test = test.join(matrix, rsuffix='_'+feature)
    return trainX, validationSetX, test

In [26]:
trainX, validationSetX, test = oneHot(trainX, validationSetX, test)

In [27]:
trainX

,TransactionDT,TransactionAmt,card1,card2,addr1,C13,P_emaildomain,C1,card5,C6,...,american express,discover,mastercard,nan_card4,visa,charge card,credit,debit,debit or credit,nan_card6
0,86400,68.500,13926,0.0,0.017809,1.0,0.029538,1.0,142.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,86401,29.000,2755,404.0,0.025426,1.0,0.043542,1.0,102.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,86469,59.000,4663,490.0,0.031955,1.0,0.094584,1.0,166.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
3,86499,50.000,18132,567.0,0.031652,25.0,0.022757,2.0,117.0,4.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,86506,50.000,4497,514.0,0.032672,1.0,0.043542,1.0,102.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539999,14238592,59.000,14128,170.0,0.025426,18.0,0.022757,1.0,219.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
540000,14238604,13.772,5812,408.0,0.117813,4.0,0.052950,4.0,224.0,2.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
540001,14238622,26.191,15944,266.0,0.117813,1.0,0.043542,1.0,137.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
540002,14238636,57.950,17188,321.0,0.024986,2.0,0.022757,2.0,226.0,2.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [28]:
validationSetX

,TransactionDT,TransactionAmt,card1,card2,addr1,C13,P_emaildomain,C1,card5,C6,...,american express,discover,mastercard,nan_card4,visa,charge card,credit,debit,debit or credit,nan_card6
540004,14238660,36.95,7207,111.0,0.026654,63.0,0.022757,9.0,226.0,6.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
540005,14238667,21.95,18132,567.0,0.016623,2.0,0.031187,1.0,117.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
540006,14238672,49.00,17321,111.0,0.026654,2.0,0.022757,1.0,226.0,2.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
540007,14238694,67.95,14181,555.0,0.038833,73.0,0.021811,10.0,224.0,5.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
540008,14238717,24.00,13780,298.0,0.025592,63.0,0.029538,1.0,226.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590535,15811047,49.00,6550,0.0,0.028598,3.0,0.029538,2.0,226.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
590536,15811049,39.50,10444,225.0,0.026654,1.0,0.043542,1.0,224.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
590537,15811079,30.95,12037,595.0,0.028665,1.0,0.043542,1.0,224.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
590538,15811088,117.00,7826,481.0,0.019177,5.0,0.021811,1.0,224.0,3.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [29]:
test

,TransactionDT,TransactionAmt,card1,card2,card3,card5,addr1,addr2,dist1,P_emaildomain,...,american express,discover,mastercard,nan_card4,visa,charge card,credit,debit,debit or credit,nan_card6
0,18403224,31.950,10409,111.0,150.0,226.0,NaN,NaN,1.0,0.043542,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1,18403263,49.000,4272,111.0,150.0,226.0,NaN,NaN,4.0,0.021811,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,18403310,171.000,4476,574.0,150.0,226.0,NaN,NaN,2635.0,0.052950,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
3,18403310,284.950,10989,360.0,150.0,166.0,NaN,NaN,17.0,0.043542,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,18403317,67.950,18018,452.0,150.0,117.0,NaN,NaN,6.0,0.043542,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
506686,34214279,94.679,13832,375.0,185.0,224.0,NaN,NaN,0.0,0.043542,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
506687,34214287,12.173,3154,408.0,185.0,224.0,0.117813,0.117813,0.0,0.052950,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
506688,34214326,49.000,16661,490.0,150.0,226.0,NaN,NaN,0.0,0.052950,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
506689,34214337,202.000,16621,516.0,150.0,224.0,NaN,NaN,0.0,0.052950,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [30]:
test.fillna(0, inplace=True)

In [31]:
trainX.to_csv('data/Parte3/trainXParte3M1.csv',index=False)
trainY.to_csv('data/Parte3/trainYParte3M1.csv',index=False)
validationSetX.to_csv('data/Parte3/validationSetXParte3M1.csv',index=False)
validationSetY.to_csv('data/Parte3/validationSetYParte3M1.csv',index=False)
test.to_csv('data/Parte3/testParte3M1.csv',index=False)

# XGBOOST

In [2]:
trainX = pd.read_csv('data/Parte3/trainXParte3M1.csv')
trainY = pd.read_csv('data/Parte3/trainYParte3M1.csv')
validationSetX = pd.read_csv('data/Parte3/validationSetXParte3M1.csv')
validationSetY = pd.read_csv('data/Parte3/validationSetYParte3M1.csv')

In [7]:
model = xgb.XGBClassifier(objective ='binary:logistic', random_state=2209, use_label_encoder=False, eval_metric = 'auc')
parameters = {'learning_rate' : [0.1, 0.15], 'max_depth' : [4,8], 'gamma' : [1, 5],
              'subsample' : [0.4, 0.5], 'colsample_bytree' : [0.3, 0.4], 'n_estimators' : [100, 200], 'alpha' : [1,5], 'lambda' : [5,8]} 
clf = RandomizedSearchCV(model, parameters, n_iter=4, n_jobs=2, verbose=100, cv=3, random_state=2209, scoring='roc_auc')
clf.fit(trainX.values, trainY.values)

Fitting 3 folds for each of 4 candidates, totalling 12 fits


RandomizedSearchCV(cv=3,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           enable_categorical=False,
                                           eval_metric='auc', gamma=None,
                                           gpu_id=None, importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate=None,
                                           max_delta_step=None, max_depth=None,
                                           min_child_weight=None, missing=nan,
                                           mo...
                                           scale_pos_weight=None,
                                           subsample=None, tree_method=

In [8]:
# descargo modelo
import pickle
with open('data/Parte3/model_backupM1', 'wb') as model_backup_file:
    pickle.dump(clf, model_backup_file)

In [3]:

import pickle
with open('data/Parte3/model_backupM1', 'rb') as file:
    clf = pickle.load(file)


In [9]:
clf.best_score_

0.8226089963561112

In [10]:
clf.best_params_

{'subsample': 0.4,
 'n_estimators': 200,
 'max_depth': 4,
 'learning_rate': 0.1,
 'lambda': 5,
 'gamma': 1,
 'colsample_bytree': 0.3,
 'alpha': 5}

In [11]:
preds_proba = clf.predict_proba(validationSetX)
preds_proba

array([[0.9885901 , 0.01140989],
       [0.9951529 , 0.00484712],
       [0.9913682 , 0.00863185],
       ...,
       [0.99248314, 0.00751684],
       [0.92500025, 0.07499974],
       [0.9620853 , 0.03791471]], dtype=float32)

In [12]:
roc_auc_score(validationSetY, preds_proba[:,1])

0.8905848105377197

In [13]:
# este NO resultó ser el mejor modelo y por eso esta parte esta comentada

#test = pd.read_csv('data/Parte3/testParte3M1.csv')

In [14]:
#test.set_index(pd.read_csv('data/test_transaction.csv')['TransactionID'],inplace=True)

In [16]:
#pd.DataFrame(clf.predict_proba(test)[:,1], index=test.index ).rename(columns={0:'isFraud'}).to_csv('data/Parte3/late_submissionM1.csv')